In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, optimization
import os 
from datasets import load_dataset
import pandas as pd
from copy import deepcopy
from torch.optim import AdamW

C:\Users\Yoan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [3]:
dream_checkpoint = "RicoBorra/DREAM-t5-small"

dream_tokenizer = AutoTokenizer.from_pretrained(dream_checkpoint)
dream_model = AutoModelForSeq2SeqLM.from_pretrained(dream_checkpoint)
dream_data_collator = DataCollatorForSeq2Seq(tokenizer=dream_tokenizer, model=dream_checkpoint)

## FLUTE Data extraction and processing
Using the few instructions from the Git Readme to have the same initial data

In [4]:
# Only the train split is available on HuggingFace
dataset = load_dataset("ColumbiaNLP/FLUTE")

In [5]:
def add_combined_cols(entry):
    
    premise = entry["premise"].strip()
    hypothesis = entry["hypothesis"].strip()
    
    if not premise.endswith("."):
        premise += "."
    assert(premise.endswith("."))
    if not hypothesis.endswith("."):
        hypothesis += "."
    assert(hypothesis.endswith("."))
    
    entry["premise_hypothesis"] = 'Premise: ' + premise + ' Hypothesis: ' + hypothesis + 'Is there a contradiction or entailment between the premise and hypothesis ?'
    entry["label_explanation"] = 'Label: ' + entry["label"] + '. Explanation: ' + entry["explanation"]
    return entry
# combine columns
combined_cols_dataset = dataset['train'].map(add_combined_cols)

# create train test split because given data has only train data
# splits are shuffled by default
dataset_train_test = combined_cols_dataset.train_test_split(test_size=0.2, seed=42)

In [6]:
def preprocess_dataset(examples):
    model_inputs = tokenizer(examples['premise_hypothesis'])
    labels = tokenizer(examples['label_explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [7]:
tokenized_ds = dataset_train_test.map(preprocess_dataset, batched=True)
tokenized_ds = tokenized_ds.remove_columns(dataset_train_test['train'].column_names)

Map:   0%|          | 0/6027 [00:00<?, ? examples/s]

Map: 100%|██████████| 1507/1507 [00:00<00:00, 2916.41 examples/s]


In [8]:
import evaluate
rouge = evaluate.load("rouge")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## System 1 : Normal classifier

In [9]:
tokenizer_s1 = deepcopy(tokenizer)
model_s1 = deepcopy(model)

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir="D:\Documents\PoliTo\Deep NLP\Project\S1Model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    seed=42,
    #weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    load_best_model_at_end=True,
    #eval_accumulation_steps=8,
    #fp16=True,
    #push_to_hub=True,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    lr_scheduler_type='linear'
)

'''The 50 first elements are chosen just to test quickly the trainer'''
trainer = Seq2SeqTrainer(
    model=model_s1,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer_s1,
    data_collator=data_collator,
    #compute_metrics=compute_metrics
)

trainer.train()

  6%|▌         | 501/9042 [00:57<18:35,  7.65it/s]

{'loss': 2.3956, 'learning_rate': 4.723512497235125e-05, 'epoch': 0.17}


 11%|█         | 1001/9042 [02:03<15:41,  8.54it/s]

{'loss': 2.0239, 'learning_rate': 4.44702499447025e-05, 'epoch': 0.33}


 17%|█▋        | 1501/9042 [03:08<16:44,  7.51it/s]

...

                                                   
100%|██████████| 9042/9042 [19:47<00:00,  9.25it/s]

{'eval_loss': 1.5241432189941406, 'eval_runtime': 23.6921, 'eval_samples_per_second': 63.608, 'eval_steps_per_second': 31.825, 'epoch': 3.0}


100%|██████████| 9042/9042 [19:48<00:00,  7.61it/s]

{'train_runtime': 1188.4944, 'train_samples_per_second': 15.213, 'train_steps_per_second': 7.608, 'train_loss': 1.7722833359618546, 'epoch': 3.0}


TrainOutput(global_step=9042, training_loss=1.7722833359618546, metrics={'train_runtime': 1188.4944, 'train_samples_per_second': 15.213, 'train_steps_per_second': 7.608, 'train_loss': 1.7722833359618546, 'epoch': 3.0})

In [16]:
i = "Premise : I was crushed by an elephant today. Hypothesis : I really never wanted to be crushed by an elephant."
t = tokenizer_s1(i, return_tensors='pt').input_ids
t = t.to(model_s1.device)
o = model_s1.generate(t, max_new_tokens = 100)
d = tokenizer_s1.decode(o[0])
d 

'<pad> Label : Contradiction. Explanation: To be crushed by an elephant is not a good thing because it is not something to be crushed by.</s>'